# HW7 DATASCI W261: Machine Learning at Scale  
 
* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  9
* **Homework Number:**  7
* **Date:**  7/10/16

<h1 style="color:#021353;">General Description</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
In this assignment you will explore networks and develop MRJob code for 
finding shortest path graph distances. To build up to large data 
you will develop your code on some very simple, toy networks.
After this you will take your developed code forward and modify it and 
apply it to two larger datasets (performing EDA along the way).

<h3>Undirected toy network dataset</h3>


In an undirected network all links are symmetric, 
i.e., for a pair of nodes 'A' and 'B,' both of the links:

A -> B and B -> A

will exist. 

The toy data are available in a sparse (stripes) representation:

(node) \t (dictionary of links)

on AWS/Dropbox via the url:

s3://ucb-mids-mls-networks/undirected_toy.txt
On under the Data Subfolder for HW7 on Dropbox with the same file name. 
The Data folder is in: https://db.tt/Kxu48mL1)

In the dictionary, target nodes are keys, link weights are values 
(here, all weights are 1, i.e., the network is unweighted).


<h3>Directed toy network dataset</h3>

In a directed network all links are not necessarily symmetric, 
i.e., for a pair of nodes 'A' and 'B,' it is possible for only one of:

A -> B or B -> A

to exist. 

These toy data are available in a sparse (stripes) representation:

(node) \t (dictionary of links)

on AWS/Dropbox via the url:

s3://ucb-mids-mls-networks/directed_toy.txt
Or under the Data Subfolder for HW7 on Dropbox with the same file name
(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

In the dictionary, target nodes are keys, link weights are values 
(here, all weights are 1, i.e., the network is unweighted).
</pre>
</div>

In [1]:
%%writefile directed_toy.txt
1	{'2': 1, '6': 1}
2	{'1': 1, '3': 1, '4': 1}
3	{'2': 1, '4': 1}
4	{'2': 1, '5': 1}
5	{'1': 1, '2': 1, '4': 1}

Overwriting directed_toy.txt


In [26]:
%%writefile d2_toy.txt
A	{'B': 1, 'E': 1000}
B	{'C': 1}
C	{'D': 1}
D	{'E': 1}
E	{'F': 1}

Overwriting d2_toy.txt


In [30]:
%%writefile d3_toy.txt
A	{'B': 10, 'C': 5}
B	{'C': 2, 'D': 1}
C	{'B': 3, 'D': 9, 'E': 2}
D	{'E': 4}
E	{'A': 7, 'D': 6}

Writing d3_toy.txt


In [1]:
%%writefile d4_toy.txt
1	{"2": 1, "5": 100}
2	{"1": 1, "3": 1, "5": 1, "4": 1}
3	{"2": 1, "4": 1}
4	{"3": 1, "2": 1, "5": 1}
5	{"1": 100, "2": 1, "4": 1, "6":1}
6	{"5": 1, "7": 1}
7	{"6": 1}

Writing d4_toy.txt


In [2]:
%%writefile undirected_toy.txt
1	{'2': 1,'5': 1}
2	{'1': 1,'3': 1,'4': 1,'5': 1}
3	{'2': 1, '4': 1}
4	{'2': 1,'3': 1,'5': 1}
5	{'1': 1, '2': 1, '4': 1}

Overwriting undirected_toy.txt


<h1 style="color:#021353;">HW 7.0: Shortest path graph distances (toy networks)</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
In this part of your assignment you will develop the base of your code for the week.

Write MRJob classes to find shortest path graph distances, as described in the lectures. In addition to finding the distances, your code should also output a distance-minimizing path between the source and target.
Work locally for this part of the assignment, and use both of the undirected and directed toy networks.

To proof you code's function, run the following jobs

- shortest path in the undirected network from node 1 to node 4
Solution: 1,5,4. NOTE: There is another shortest path also (HINT: 1->5->4)! Either will suffice (you will find this also in the remaining problems. E.g., 7.2 and 7.4.
 

- shortest path in the directed network from node 1 to node 5
Solution: 1,2,4,5

and report your output---make sure it is correct!

<h3>Main dataset 1: NLTK synonyms</h3>

In the next part of this assignment you will explore a network derived from the NLTK synonym database used for evaluation in HW 5. At a high level, this network is undirected, defined so that there exists link between two nodes/words if the pair or words are a synonym. These data may be found at the location:

<a href="s3://ucb-mids-mls-networks/synNet/synNet.txt">s3://ucb-mids-mls-networks/synNet/synNet.txt</a>
<a href="s3://ucb-mids-mls-networks/synNet/indices.txt">s3://ucb-mids-mls-networks/synNet/indices.txt</a>
On under the Data Subfolder for HW7 on Dropbox with the same file names

where synNet.txt contains a sparse representation of the network:

(index) \t (dictionary of links)

in indexed form, and indices.txt contains a lookup list

(word) \t (index)

of indices and words. This network is small enough for you to explore and run
scripts locally, but will also be good for a systems test (for later) on AWS.

In the dictionary, target nodes are keys, link weights are values 
(here, all weights are 1, i.e., the network is unweighted).
</pre>
</div>

In [4]:
%%writefile SSSP.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
import os

class MRSSSP(MRJob):
    def configure_options(self):
        # Configure a new command line option to enable the job to accept the value
        # of k required by the Kmeans algorithm.
        super(MRSSSP, self).configure_options()
        self.add_passthrough_option('--k', type='int', default=4)
    
    #load data and output the nearest centroid index and data point plus a count of 1. 
    def mapper(self, _, line):
        node_num, adj_dict, min_dist, short_path, state = line.strip().split('\t')
        if state == 'Q':
            yield node_num, tuple((adj_dict, min_dist, short_path+node_num, 'V'))
            d = eval(adj_dict)
            for node, dist in d.iteritems():
                yield node, tuple(('{}', float(min_dist)+float(dist), short_path+node_num, 'Q'))
        else:
            yield node_num, tuple((adj_dict, min_dist, short_path, state))
    
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, node, data):
        f_adj_dict = Counter()
        f_min_dist = 1000000.0
        f_short_path = ''
        f_state = 'U'
        #print node
        for adj_dict, min_dist, short_path, state in data:
            #print adj_dict, min_dist, short_path, state
            f_adj_dict += Counter(eval(adj_dict))
            if float(min_dist) < f_min_dist:
                f_min_dist = float(min_dist)
                f_short_path = short_path
                f_state = state
        yield node, tuple((f_adj_dict, f_min_dist, f_short_path, f_state))

    def steps(self):
        # Create the steps for the MRJob.  There is only one MRJob here containing
        # a mapper, a combiner and a reducer.
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer)
               ]
            
if __name__ == '__main__':
    MRSSSP.run()

Overwriting SSSP.py


In [5]:
# These are helper functions for the shortest path code

def init_work_table(ifilename, ofilename):
    with open(ifilename, 'r') as ifile, open(ofilename, 'w') as ofile:
        line = ifile.readline()
        ofile.write('%s\t%f\t%s\t%s\n' % (line.strip(),0.0,"",'Q'))
        for line in ifile.readlines():
            # ??fix this to be infinity or something
            ofile.write('%s\t%f\t%s\t%s\n' % (line.strip(),1000000.0,"",'U'))

In [6]:
%reload_ext autoreload
%autoreload 2
from numpy import random
from SSSP import MRSSSP

work_filename = 'work_table.txt'
init_work_table('d4_toy.txt', work_filename)

# Run the KMeans MRJob
mr_job = MRSSSP(args=['work_table.txt'])
    
# Update centroids iteratively
i = 0
not_all_visited = True
while(not_all_visited):
    work_table = {}
    not_all_visited = False
    print('Iteration %d' % (i))
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key, value
            work_table[key] = value
            if value[3] in 'QU':
                not_all_visited = True 
        
        # Update work_table for the next iteration
        with open(work_filename, 'w') as f:
            for key, value in work_table.iteritems():
                f.write(key)
                f.writelines('\t'+str(v) for v in value)
                f.write('\n')
    i += 1

Iteration 0
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 100.0, u'1', u'Q']
6 [{u'5': 1, u'7': 1}, 1000000.0, u'', u'U']
7 [{u'6': 1}, 1000000.0, u'', u'U']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'1', u'Q']
3 [{u'2': 1, u'4': 1}, 1000000.0, u'', u'U']
4 [{u'3': 1, u'2': 1, u'5': 1}, 1000000.0, u'', u'U']
Iteration 1
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 2.0, u'12', u'Q']
6 [{u'5': 1, u'7': 1}, 101.0, u'15', u'Q']
7 [{u'6': 1}, 1000000.0, u'', u'U']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'12', u'V']
3 [{u'2': 1, u'4': 1}, 2.0, u'12', u'Q']
4 [{u'3': 1, u'2': 1, u'5': 1}, 2.0, u'12', u'Q']
Iteration 2
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 2.0, u'125', u'V']
6 [{u'5': 1, u'7': 1}, 3.0, u'125', u'Q']
7 [{u'6': 1}, 102.0, u'156', u'Q']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'12', u'V']
3 [{u'2': 1, u'4': 1}, 2.0, u'123', u'V']
4 [{u'3'

In [7]:
!cat work_table.txt

1	{u'2': 1, u'5': 100}	0.0	1	V
3	{u'2': 1, u'4': 1}	2.0	123	V
2	{u'1': 1, u'3': 1, u'5': 1, u'4': 1}	1.0	12	V
5	{u'1': 100, u'2': 1, u'4': 1, u'6': 1}	2.0	125	V
4	{u'3': 1, u'2': 1, u'5': 1}	2.0	124	V
7	{u'6': 1}	4.0	12567	V
6	{u'5': 1, u'7': 1}	3.0	1256	V


In [8]:
#!mv work_table.txt work_table_undirected_final.txt
#!mv work_table.txt work_table_directed_final.txt   
#!mv work_table.txt work_table_d2_final.txt 
!mv work_table.txt work_table_d4_final.txt 
!cat work_table_undirected_final.txt
!echo
!cat work_table_directed_final.txt   
!echo
!cat work_table_d2_final.txt  
!echo
!cat work_table_d3_final.txt  
!echo
!cat work_table_d4_final.txt

1	{'2': 1,'5': 1}	0.000000	1	V
3	{u'2': 1, u'4': 1}	2.0	123	V
2	{u'1': 1, u'3': 1, u'5': 1, u'4': 1}	1.0	12	V
5	{u'1': 1, u'2': 1, u'4': 1}	1.0	15	V
4	{u'3': 1, u'2': 1, u'5': 1}	2.0	124	V

1	{'2': 1, '6': 1}	0.000000	1	V
3	{u'2': 1, u'4': 1}	2.0	123	V
2	{u'1': 1, u'3': 1, u'4': 1}	1.0	12	V
5	{u'1': 1, u'2': 1, u'4': 1}	3.0	1245	V
4	{u'2': 1, u'5': 1}	2.0	124	V
6	{}	1.0	16	V

A	{u'B': 1, u'E': 1000}	0.0	A	V
C	{u'D': 1}	2.0	ABC	V
B	{u'C': 1}	1.0	AB	V
E	{u'F': 1}	4.0	ABCDE	V
D	{u'E': 1}	3.0	ABCD	V
F	{}	5.0	ABCDEF	V

A	{u'C': 5, u'B': 10}	0.0	A	V
C	{u'B': 3, u'E': 2, u'D': 9}	5.0	AC	V
B	{u'C': 2, u'D': 1}	8.0	ACB	V
E	{u'A': 7, u'D': 6}	7.0	ACE	V
D	{u'E': 4}	9.0	ACBD	V

1	{u'2': 1, u'5': 100}	0.0	1	V
3	{u'2': 1, u'4': 1}	2.0	123	V
2	{u'1': 1, u'3': 1, u'5': 1, u'4': 1}	1.0	12	V
5	{u'1': 100, u'2': 1, u'4': 1, u'6': 1}	2.0	125	V
4	{u'3': 1, u'2': 1, u'5': 1}	2.0	124	V
7	{u'6': 1}	4.0	12567	V
6	{u'5': 1, u'7': 1}	3.0	1256	V


<h1 style="color:#021353;">HW 7.1: Exploratory data analysis (NLTK synonyms)</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Using MRJob, explore the synonyms network data.
Consider plotting the degree distribution (does it follow a power law?),
and determine some of the key features, like:

number of nodes, 
number links,
or the average degree (i.e., the average number of links per node),
etc...

As you develop your code, please be sure to run it locally first (though on the whole dataset). 
Once you have gotten you code to run locally, deploy it on AWS as a systems test
in preparation for our next dataset (which will require AWS).
</pre>
</div>


In [4]:
%%writefile SSSP.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
import os

class MRSSSP(MRJob):
    def configure_options(self):
        # Configure a new command line option to enable the job to accept the value
        # of k required by the Kmeans algorithm.
        super(MRSSSP, self).configure_options()
        self.add_passthrough_option('--k', type='int', default=4)
    
    #load data and output the nearest centroid index and data point plus a count of 1. 
    def mapper(self, _, line):
        node_num, adj_dict, min_dist, short_path, state = line.strip().split('\t')
        if state == 'Q':
            yield node_num, tuple((adj_dict, min_dist, short_path+node_num, 'V'))
            d = eval(adj_dict)
            for node, dist in d.iteritems():
                yield node, tuple(('{}', float(min_dist)+float(dist), short_path+node_num, 'Q'))
        else:
            yield node_num, tuple((adj_dict, min_dist, short_path, state))
    
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, node, data):
        f_adj_dict = Counter()
        f_min_dist = 1000000.0
        f_short_path = ''
        f_state = 'U'
        #print node
        for adj_dict, min_dist, short_path, state in data:
            #print adj_dict, min_dist, short_path, state
            f_adj_dict += Counter(eval(adj_dict))
            if float(min_dist) < f_min_dist:
                f_min_dist = float(min_dist)
                f_short_path = short_path
                f_state = state
        yield node, tuple((f_adj_dict, f_min_dist, f_short_path, f_state))

    def steps(self):
        # Create the steps for the MRJob.  There is only one MRJob here containing
        # a mapper, a combiner and a reducer.
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer)
               ]
            
if __name__ == '__main__':
    MRSSSP.run()

Overwriting SSSP.py


In [5]:
# These are helper functions for the shortest path code

def init_work_table(ifilename, ofilename):
    with open(ifilename, 'r') as ifile, open(ofilename, 'w') as ofile:
        line = ifile.readline()
        ofile.write('%s\t%f\t%s\t%s\n' % (line.strip(),0.0,"",'Q'))
        for line in ifile.readlines():
            # ??fix this to be infinity or something
            ofile.write('%s\t%f\t%s\t%s\n' % (line.strip(),1000000.0,"",'U'))

In [6]:
%reload_ext autoreload
%autoreload 2
from numpy import random
from SSSP import MRSSSP

work_filename = 'work_table.txt'
init_work_table('d4_toy.txt', work_filename)

# Run the KMeans MRJob
mr_job = MRSSSP(args=['work_table.txt'])
    
# Update centroids iteratively
i = 0
not_all_visited = True
while(not_all_visited):
    work_table = {}
    not_all_visited = False
    print('Iteration %d' % (i))
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key, value
            work_table[key] = value
            if value[3] in 'QU':
                not_all_visited = True 
        
        # Update work_table for the next iteration
        with open(work_filename, 'w') as f:
            for key, value in work_table.iteritems():
                f.write(key)
                f.writelines('\t'+str(v) for v in value)
                f.write('\n')
    i += 1

Iteration 0
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 100.0, u'1', u'Q']
6 [{u'5': 1, u'7': 1}, 1000000.0, u'', u'U']
7 [{u'6': 1}, 1000000.0, u'', u'U']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'1', u'Q']
3 [{u'2': 1, u'4': 1}, 1000000.0, u'', u'U']
4 [{u'3': 1, u'2': 1, u'5': 1}, 1000000.0, u'', u'U']
Iteration 1
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 2.0, u'12', u'Q']
6 [{u'5': 1, u'7': 1}, 101.0, u'15', u'Q']
7 [{u'6': 1}, 1000000.0, u'', u'U']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'12', u'V']
3 [{u'2': 1, u'4': 1}, 2.0, u'12', u'Q']
4 [{u'3': 1, u'2': 1, u'5': 1}, 2.0, u'12', u'Q']
Iteration 2
5 [{u'1': 100, u'2': 1, u'4': 1, u'6': 1}, 2.0, u'125', u'V']
6 [{u'5': 1, u'7': 1}, 3.0, u'125', u'Q']
7 [{u'6': 1}, 102.0, u'156', u'Q']
1 [{u'2': 1, u'5': 100}, 0.0, u'1', u'V']
2 [{u'1': 1, u'3': 1, u'5': 1, u'4': 1}, 1.0, u'12', u'V']
3 [{u'2': 1, u'4': 1}, 2.0, u'123', u'V']
4 [{u'3'

In [7]:
!cat work_table.txt

1	{u'2': 1, u'5': 100}	0.0	1	V
3	{u'2': 1, u'4': 1}	2.0	123	V
2	{u'1': 1, u'3': 1, u'5': 1, u'4': 1}	1.0	12	V
5	{u'1': 100, u'2': 1, u'4': 1, u'6': 1}	2.0	125	V
4	{u'3': 1, u'2': 1, u'5': 1}	2.0	124	V
7	{u'6': 1}	4.0	12567	V
6	{u'5': 1, u'7': 1}	3.0	1256	V


<h1 style="color:#021353;">HW 7.2: Shortest path graph distances (NLTK synonyms)</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Write (reuse your code from 7.0) an MRJob class to find shortest path graph distances, 
and apply it to the NLTK synonyms network dataset. 

Proof your code's function by running the job:

- shortest path starting at "walk" (index=7827) and ending at "make" (index=536),

and showing you code's output. Once again, your output should include the path and the distance.

As you develop your code, please be sure to run it locally first (though on the whole dataset). 
Once you have gotten you code to run locally, deploy it on AWS as a systems test
in preparation for our next dataset (which will require AWS).

=====================================
<strong>NOTE: Dataset 2 English Wikipedia hyperlink network.data </strong>
The dataset is available via Dropbox at:

https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0

on S3 at 
<a href="s3://ucb-mids-mls-networks/wikipedia/">s3://ucb-mids-mls-networks/wikipedia/</a>
<a href="s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt">s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt</a> # Graph
<a href="s3://ucb-mids-mls-networks/wikipedia/indices.txt">s3://ucb-mids-mls-networks/wikipedia/indices.txt</a> # Page titles and page Ids

For the remainder of this assignment you will explore the English Wikipedia hyperlink network.

The dataset is built from the Sept. 2015 XML snapshot of English Wikipedia.
For this directed network, a link between articles: 

A -> B

is defined by the existence of a hyperlink in A pointing to B.
This network also exists in the indexed format:

Data: <a href="s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt">s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt</a>
Data: <a href="s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-in.txt">s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-in.txt</a>
Data: <a href="s3://ucb-mids-mls-networks/wikipedia/indices.txt">s3://ucb-mids-mls-networks/wikipedia/indices.txt</a>

but has an index with more detailed data:

(article name) \t (index) \t (in degree) \t (out degree)

In the dictionary, target nodes are keys, link weights are values .
Here, a weight indicates the number of time a page links to another.
However, for the sake of this assignment, treat this an unweighted network,
and set all weights to 1 upon data input.

</pre>
</div>

<h1 style="color:#021353;">HW 7.3: Exploratory data analysis (Wikipedia)</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Using MRJob, explore the Wikipedia network data on the AWS cloud. Reuse your code from HW 7.1---does is scale well? 

Be cautioned that Wikipedia is a directed network, where links are not symmetric. 
So, even though a node may be linked to, it will not appear as a primary record itself if it has no out-links. 

This means that you may have to ADJUST your code (depending on its design). 

To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.
</pre>
</div>

<h1 style="color:#021353;">HW 7.4: Shortest path graph distances (Wikipedia)</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Using MRJob, find shortest path graph distances in the Wikipedia network on the AWS cloud.
Reuse your code from 7.2, but once again be warned of Wikipedia being a directed network.
To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.

When running your code on the Wikipedia network, proof its function by running the job:

- shortest path from "Ireland" (index=6176135) to "University of California, Berkeley" (index=13466359),

and show your code's output. Show the shortest path in terms of just page IDS but also in terms of the name of page (show of your MapReduce join skills!!)

Once your code is running, find some other shortest paths and report your results.
</pre>
</div>

<h1 style="color:#021353;">HW 7.5: Conceptual exercise: Largest single-source network distances</h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Suppose you wanted to find the largest network distance from a single source,
i.e., a node that is the furthest (but still reachable) from a single source.

How would you implement this task? 
How is this different from finding the shortest path graph distances?

Is this task more difficult to implement than the shortest path distance?

As you respond, please comment on program structure, runtimes, iterations, general system requirements, etc...
</pre>
</div>

<h1 style="color:#021353;">HW 7.5.1: </h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Can we utilize combiners in the HW 7 to perform the shortest path implementation?
Does order inversion help with the HW 7 shortest path implementation?
</pre>
</div>

<h1 style="color:#021353;">HW 7.5.2: OPTIONAL </h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Implement combiners in the context of HW 7.5 and contrast the performance of this implementation versus the implementation with no combiners. 

Please report the cluster configuration and runtimes in tabular format for both experiments and comment on your findings.
</pre>
</div>

<h1 style="color:#021353;">HW 7.6: Computational exercise: Largest single-source network distances: OPTIONAL </h1>
<div style="margin:10px;border-left:5px solid #eee;">
<pre style="font-family:sans-serif;background-color:transparent">
Using MRJob, write a code to find the largest graph distance and distance-maximizing nodes from a single-source.
Test your code first on the toy networks and synonyms network to proof its function.
</pre>
</div>

==================END HW 7==================